In [1]:
import sys

print(sys.getsizeof([0] * 512))


4152


In [2]:
print(sys.getsizeof("a"))
print(sys.getsizeof(["a"] * 512))

42
4152


In [3]:
ptable = [ i.bit_count() for i in range(1 << 9) ]

In [4]:
b = 0b100010010
print(ptable[b])
print(b.bit_count())

3
3


In [5]:
%timeit ptable[b]
%timeit b.bit_count()

22.7 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
31.4 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
ptable = [ i.bit_count() for i in range(1 << 16) ]

def popcount5(b):
    count = 0
    while b:
        count += ptable[b & 0b1111111111111111]
        b >>= 16
    return count

In [7]:
for i in [10, 20, 32, 50, 100, 1000]:
    b = (1 << i) - 1
    print(f"i = {i}")
    print(popcount5(b))
    %timeit popcount5(b)
    print(b.bit_count())
    %timeit b.bit_count()

i = 10
10
139 ns ± 0.289 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
10
31.9 ns ± 0.89 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 20
20
211 ns ± 1.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
20
31.4 ns ± 0.199 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 32
32
253 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
32
33 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 50
50
439 ns ± 1.78 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
50
32.7 ns ± 0.278 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 100
100
791 ns ± 6.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
100
35.8 ns ± 0.134 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 1000
1000
8.99 μs ± 133 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1000
92.6 ns ± 0.275 ns per loop (mean ± std. dev. of 7 runs

In [8]:
def popcount32(b):
    return ptable[b & 0b1111111111111111] + ptable[b >> 16]

In [9]:
for i in [10, 20, 32]:
    b = (1 << i) - 1
    print(f"i = {i}")
    print(popcount32(b))
    %timeit popcount32(b)

i = 10
10
120 ns ± 0.246 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 20
20
120 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
i = 32
32
143 ns ± 0.882 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
from marubatsu import BitBoard

class BitBoard3x3(BitBoard):
   pass

In [11]:
def calc_legal_moves(self):
    # マークが配置されているマスのビットが 1 になるビットボードを計算する
    board = self.board[0] | self.board[1]
    # 空のマスのビットが 1 になるビットボードを計算する
    board = 0b111111111 - board
    legal_moves = []
    # board が 0 になるまで繰り返し処理を行う
    while board:
        # 次の合法手である LOB を計算する
        move = board & (-board)
        legal_moves.append(move)
        # board の RSB を 0 にして move のデータを削除する
        board -= move

    return legal_moves

BitBoard3x3.calc_legal_moves = calc_legal_moves

def calc_same_hashables(self, move=None):
    if move is None:
        hashables = set()
    else:
        hashables = {}
    if move is not None:
        x, y = self.move_to_xy(move)
    circlebb = self.board[0]
    crossbb = self.board[1]
    for _ in range(4):
        # 左右の反転処理
        c = (circlebb ^ (circlebb >> 6)) & 0b111
        circlebb = c ^ (c << 6) ^ circlebb        
        c = (crossbb ^ (crossbb >> 6)) & 0b111
        crossbb = c ^ (c << 6) ^ crossbb        
        hashable = circlebb | (crossbb << self.bit_length) 
        if move is None:
            hashables.add(hashable)
        else:
            x = self.BOARD_SIZE - x - 1
            hashables[hashable] = self.xy_to_move(x, y)
            
        # 転地処理
        c = (circlebb ^ (circlebb >> 2)) & 0b100010
        circlebb = c ^ (c << 2) ^ circlebb 
        c = (circlebb ^ (circlebb >> 4)) & 0b100
        circlebb = c ^ (c << 4) ^ circlebb 
        c = (crossbb ^ (crossbb >> 2)) & 0b100010
        crossbb = c ^ (c << 2) ^ crossbb 
        c = (crossbb ^ (crossbb >> 4)) & 0b100
        crossbb = c ^ (c << 4) ^ crossbb 
        hashable = circlebb | (crossbb << self.bit_length) 
        if move is None:
            hashables.add(hashable)
        else:
            x, y = y, x
            hashables[hashable] = self.xy_to_move(x, y)
    return hashables  

BitBoard3x3.calc_legcalc_same_hashables = calc_same_hashables

In [12]:
def __init__(self, *args, **kwargs):
    self.BOARD_SIZE = 3
    self.bit_length = 9
    self.board = [0, 0]
        
BitBoard3x3.__init__ = __init__

def setmark_by_move(self, move, mark):
    if mark == self.EMPTY:
        self.board[self.CIRCLE] &= ~0 ^ move
        self.board[self.CROSS] &= ~0 ^ move
    else:
        self.board[mark] |= move
        
BitBoard3x3.setmark_by_move = setmark_by_move

def is_winner(self, player, last_move):
    x, y = self.move_to_xy(last_move)
    colmasks = [0b000000111, 0b000111000, 0b111000000]
    rowmasks = [0b001001001, 0b010010010, 0b100100100]
    diamask1 = 0b100010001
    diamask2 = 0b001010100
    colmask = colmasks[x]
    rowmask = rowmasks[y]
    board = self.board[player]
    if board & colmask == colmask or board & rowmask == rowmask:
        return True
    # 左上から右下方向の判定
    if x == y and board & diamask1 == diamask1:
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and board & diamask2 == diamask2:
        return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False    

BitBoard3x3.is_winner = is_winner

In [13]:
BitBoard3x3.ptable = [i.bit_count() for i in range(1 << 9)]
BitBoard3x3.masklist = [
    0b000000111, # 0 列のビットマスク
    0b000111000, # 1 列のビットマスク
    0b111000000, # 2 列のビットマスク
    0b001001001, # 0 行のビットマスク
    0b010010010, # 1 行のビットマスク
    0b100100100, # 2 行のビットマスク
    0b100010001, # 対角線 1 のビットマスク
    0b001010100, # 対角線 2 のビットマスク
]

In [14]:
from collections import defaultdict

def count_markpats(self, turn, last_turn):
    markpats = defaultdict(int)
    for mask in self.masklist:
        turncount = self.ptable[self.board[turn] & mask]
        lastturncount = self.ptable[self.board[last_turn] & mask]
        emptycount = self.BOARD_SIZE - turncount - lastturncount
        markpats[(lastturncount, turncount, emptycount)] += 1

    return markpats    

BitBoard3x3.count_markpats = count_markpats

In [15]:
bb1 = BitBoard()
bb2 = BitBoard3x3()

%timeit bb1.count_markpats(bb1.CIRCLE, bb1.CROSS)
%timeit bb2.count_markpats(bb2.CIRCLE, bb2.CROSS)

2.73 μs ± 11 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.62 μs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
from util import benchmark

boardclass = BitBoard3x3
print(f"boardclass: {boardclass.__name__}")
benchmark(mbparams={"boardclass": boardclass})

boardclass: BitBoard3x3
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 21890.19it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:15<00:00, 3215.41it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
  8.1 ms ±   0.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
colmasks = [0b000000111, 0b000111000, 0b111000000]
rowmasks = [0b001001001, 0b010010010, 0b100100100]
diamask1 = 0b100010001
diamask2 = 0b001010100
move_to_masklist = {}
for x in range(3):
    for y in range(3):
        move = 1 << (y + 3 * x)
        move_to_masklist[move] = [colmasks[x], rowmasks[y]]
        if x == y:
            move_to_masklist[move].append(diamask1)
        if x + y == 2:
            move_to_masklist[move].append(diamask2)

BitBoard3x3.move_to_masklist = move_to_masklist

In [18]:
from marubatsu import Marubatsu

def judge(self, last_turn, last_move, move_count):
    if move_count < 5:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    board = self.board[last_turn]
    for bitmask in self.move_to_masklist[last_move]:
        if board & bitmask == bitmask:
            return last_turn
    # 引き分けの判定
    if move_count == self.bit_length:
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING    
    
BitBoard3x3.judge = judge

In [19]:
print(f"boardclass: {boardclass.__name__}")
benchmark(mbparams={"boardclass": boardclass})

boardclass: BitBoard3x3
ai2 VS ai2


100%|██████████| 50000/50000 [00:02<00:00, 24220.47it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:15<00:00, 3238.18it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
  7.9 ms ±   1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
def popcount3(b):
    b = (b & 0x55555555) + ((b & 0xaaaaaaaa) >> 1)
    b = (b & 0x33333333) + ((b & 0xcccccccc) >> 2)
    b = (b & 0x0f0f0f0f) + ((b & 0xf0f0f0f0) >> 4)
    b = (b & 0x00ff00ff) + ((b & 0xff00ff00) >> 8)
    return (b & 0x0000ffff) + ((b & 0x0000ffff) >> 16)

In [21]:
def popcount6(x):
    x = x - ((x >> 1) & 0x55555555)
    x = (x & 0x33333333) + ((x >> 2) & 0x3333333)
    x = (x + (x >> 4)) & 0x0f0f0f0f
    x = x + (x >> 8) 
    x = x + (x >> 16) 
    return x & 0x0000007f

In [22]:
blist = [
  0b0,
  0b1,
  0b100000000,
  0b10000000000000000000000000000000,
  0b11,
  0b100000001,
  0b10000000000000000000000000000001,
  0b11111,
  0b101010101,
  0b10000001000000001000000001000001,
  0b111111111,
  0b11111111111111111111111111111111,
]

print("popcount3")
print("                            data | pc | bl | time")
for b in blist:
    popcount = popcount3(b)
    print(f"{b:32b} | {popcount:2} | {b.bit_length():2} | ", end="")
    %timeit popcount3(b)
    
print("popcount6")
print("                            data | pc | bl | time")
for b in blist:
    popcount = popcount6(b)
    print(f"{b:32b} | {popcount:2} | {b.bit_length():2} | ", end="")
    %timeit popcount6(b)

popcount3
                            data | pc | bl | time
                               0 |  0 |  0 | 441 ns ± 4.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
                               1 |  1 |  1 | 440 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
                       100000000 |  1 |  9 | 492 ns ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
10000000000000000000000000000000 |  0 | 32 | 619 ns ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
                              11 |  2 |  2 | 481 ns ± 6.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
                       100000001 |  2 |  9 | 550 ns ± 1.91 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
10000000000000000000000000000001 |  1 | 32 | 614 ns ± 1.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
                           11111 |  5 |  5 | 474 ns ± 1.04 ns per loop (mean ± std. de